本筆記根據[Tensorflow beginner notebook](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb)進行修改練習。

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

用(x_train, y_train), (x_test, y_test) = mnist.load_data()叫進來時，x_train的shape是(60000,28,28)。dtype是uint8，可能值為0,1,2,...,255。每一樣本為一28x28的灰階影像。字跡較白，背景黑色。

用x_train, x_test = x_train / 255.0, x_test / 255.0轉換後x_train的dtype是float64，每一元素的值介於0與1之間。用來做模型的Input。

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape,x_train.dtype)
print(tf.math.bincount(x_train[0]))
print(tf.reduce_sum(tf.math.bincount(x_train[0])))
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
x_train[0], x_train[0].dtype, x_train[0].shape

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

**%precison** is one of magic commands of ipython
[built-in magic commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html)

若 images 為由n個值為0與1之間的28x28影像陣列，則tf.nn.softmax(model(images).numpy()).numpy()為nx10的機率陣列，第k列為第k個影像歸類為0,1,2,...,9的機率。

In [ ]:
%precision 6
predictions = model(x_train[:1]).numpy()
predictions

In [ ]:
tf.nn.softmax(predictions).numpy()

loss_fn 為SparseCategoricalCrossentropy(from_logits=True)，$-\log(p)$

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
print(y_train[:1])
label=y_train[:1][0]
p = tf.math.exp(predictions[0][label])/tf.reduce_sum(tf.math.exp(predictions[0]))
print(-tf.math.log(tf.nn.softmax(predictions).numpy()[0][label]))
print(-tf.math.log(p))

In [ ]:
loss_fn(y_train[:1], predictions).numpy()

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=30)

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [ ]:
import numpy as np
def predict(gray28s):
    p=probability_model(gray28s)
    print(p)
    return np.argmax(p, axis=-1)

In [ ]:
predict(x_test[:10])

In [ ]:
p= probability_model(x_test[:10])
print(p)
v = np.argmax(p,axis=-1)
print(v)

In [ ]:
from matplotlib import pyplot as plt
digit=(x_train[4]*255).astype('uint8')


In [ ]:
digit


In [ ]:
plt.imshow(digit, cmap=plt.cm.gray)
plt.show()

In [ ]:
for i in range(10):
    plt.imshow((x_test[i]*255).astype('uint8'), cmap=plt.cm.gray)
    print(y_test[i], v[i])
    plt.show()

In [ ]:
import matplotlib.image as img
import cv2

In [ ]:
imgname = "b-four-fat.png"
image = img.imread(imgname)
#image.resize(28,28)
plt.imshow(image)
plt.show()

In [ ]:
myimg = cv2.imread(imgname, cv2.IMREAD_UNCHANGED)
#img = (img*255)
#print(img)
print(myimg.shape, myimg.dtype)
#print(np.max(img,axis=1))
gray = cv2.cvtColor(myimg, cv2.COLOR_BGRA2GRAY)
print(gray.shape, gray.dtype)
gray28 = cv2.resize(gray, (28,28), interpolation = cv2.INTER_AREA)
print(gray28.shape,gray28.dtype)
print(tf.math.bincount(gray28))
print(tf.reduce_sum(tf.math.bincount(gray28)))

In [ ]:
#cv2.imshow("Resized image",gray28)
#cv2.waitKey(0)
#cv2.destroyAllWindows()
plt.imshow(gray28, cmap=plt.cm.gray)
plt.show()

In [ ]:
np.array([gray28/255.0]).shape

In [ ]:
predict(np.array([gray28/255.0]))[0]

In [ ]:
plt.imshow(gray28, cmap=plt.cm.gray)
plt.show()

In [ ]:
import numpy as np
np.max(image), np.min(image)

In [ ]:
image1=(image*255).astype("uint8")

In [ ]:
plt.imshow(image1)
plt.show()

In [ ]:
np.max(image1), np.min(image1), image1.shape

用[ixartz 網頁](https://digit-recognition.ixartz.com/)製作加粗的數字，以mypredic函數辨識，只有7失敗，為什麼？

In [ ]:
def mypredict(imgname, plot=False):
    myimg = cv2.imread(imgname, cv2.IMREAD_UNCHANGED)
    gray = cv2.cvtColor(myimg, cv2.COLOR_BGRA2GRAY)
    gray28 = cv2.resize(gray, (28,28), interpolation = cv2.INTER_AREA)
    if plot:
        plt.imshow(gray28, cmap=plt.cm.gray)
        plt.show()
    d = predict(np.array([gray28/255.0]))[0]
    print(imgname, d)

In [ ]:
def mypredictblur(imgname, plot=False):
    myimg = cv2.imread(imgname, cv2.IMREAD_UNCHANGED)
    gray = cv2.cvtColor(myimg, cv2.COLOR_BGRA2GRAY)
    gray28 = cv2.resize(gray, (28,28), interpolation = cv2.INTER_AREA)
    gray28blur = cv2.blur(gray28, (3,3))
    if plot:
        plt.imshow(gray28, cmap=plt.cm.gray)
        plt.show()
        plt.imshow(gray28blur, cmap=plt.cm.gray)
        plt.show()
    d = predict(np.array([gray28blur/255.0]))[0]
    print(imgname, d)

In [ ]:
mypredict("b-seven-fat-4.png")

In [ ]:
for name in ["b-zero-fat.png","b-one-fat.png","b-two-fat.png","b-three-fat.png","b-four-fat.png","b-five-fat.png","b-six-fat.png","b-seven-fat.png","b-eight-fat.png","b-nine-fat.png"]:
    mypredict(name)

In [ ]:
for name in ["b-seven-fat.png","b-seven-fat-2.png","b-seven-fat-3.png","b-seven-fat-4.png","b-seven-fat-5.png"]:
    mypredict(name, True)

In [ ]:
for name in ["b-seven-fat.png","b-seven-fat-2.png","b-seven-fat-3.png","b-seven-fat-4.png","b-seven-fat-5.png"]:
    mypredictblur(name, True)